# 3 - Spend Analysis
Prepared by: Nickolas K. Freeman, PhD

From Wikipedia:

> Spend Analysis is the process of collecting, cleansing, classifying and analyzing expenditure data with the purpose of decreasing procurement costs, improving efficiency, and monitoring controls and compliance. It can also be leveraged in other areas of business such as inventory management, contract management, complex sourcing, supplier management, budgeting, planning, and product development.
>
> There are three core areas of spend analysis - visibility, analysis, and process. By leveraging all three, companies can generate answers to the crucial questions affecting their spending, including:
>
> - What am I really spending?
> - With whom am I spending it?
> - Am I getting what was promised for that spend?
> Spend analysis is often viewed as part of a larger domain known as spend management which incorporates spend analysis, commodity management and strategic sourcing.
> 
> Companies perform a spend analysis for several reasons. The core business driver for most organizations is profitability. In addition to improving compliance and reducing cycle times, performing detailed spend analysis helps companies find new areas of savings that previously went untapped, and hold on to past areas of savings that they have already negotiated.

In this notebook, we will use U.S. government data on contract spending available from https://usaspending.org to analyze FY 2019 expenditures in the state of Alabama. In particular, we will investigate:
- Which products and services are being purchased?
- Who is purchasing these products or services?
- From whom are the products or services being purchased?
- How are these products or services being purchased? 

We will be using Python and the `pandas` library to demonstrate a simple spend analysis. The data we will be analyzing is the same that we used in the *Pandas Overview* notebook. The data, which was downloaded from https://usaspending.org, includes records of government expenditures by the state of Alabama durng fiscal year 2019 and is provided in a *comma-separated value* format. The following code block imports the necessary libraries, loads the data, and stores it in a variable named `data`.

In [ ]:
import pathlib

import pandas as pd

# Specify maximum columns = 40
pd.set_option('display.max_columns', 40)

# Specify floating-point precision
pd.set_option('display.float_format', '{:.4f}'.format)

data_filepath = pathlib.Path('data', 'AL_FY2019.csv')

if data_filepath.exists():
    print('File exists, reading with pandas.')
    data = pd.read_csv(data_filepath)
else:
    print('File does not exist!')

## Which products/services are being purchased?

We will now proceed to investigate the first question: Which products/services are being purchased?

Let's begin by reminding ourselves of the available columns.

In [ ]:
data.columns

The North American Industry Classification System (NAICS) is the standard used by Federal agencies in classifying business establishments for the purpose of collecting, analyzing, and publishing data related to the U.S. business economy. We will use the NAICS codes provided as a proxy for the product/service type. 

We will use the groupby method of available to `pandas` `DataFrame` objects to determine the frequently purchased products/services. The following code block shows how we can use this functionality to group the data by `naics_code` and `naics_description`, counting the number of unique (`nunique`) awards for each group and summing the total dollars obligated for each group.

In [ ]:
groupby_columns = ['naics_code', 'naics_description']

agg_dict = {
    'award_id_piid':['nunique'],
    'total_dollars_obligated': ['sum'],
}

grouped_data = data.groupby(groupby_columns).agg(agg_dict)
grouped_data

Notice that the object returned is a `DataFrame`.

In [ ]:
type(grouped_data)

In this case, we have a multi-dimensional index.

In [ ]:
grouped_data.index

Our column names are also stored as a multi-dimenional index.

In [ ]:
grouped_data.columns

Personally, I like to flatten the multi-dimensional column names using the following approach.

In [ ]:
grouped_data.columns = ['_'.join(col).strip() for col in grouped_data.columns.values]
grouped_data

Due to the underlying dependence on `numpy`, calculations using the columns are easy.

In [ ]:
grouped_data['dollars_obligated_proportion'] = (grouped_data['total_dollars_obligated_sum']
                                               /grouped_data['total_dollars_obligated_sum'].sum())

grouped_data['awards_proportion'] = (grouped_data['award_id_piid_nunique']/
                                     grouped_data['award_id_piid_nunique'].sum())

grouped_data

The `nlargest` method allows us to quickly identify the subset of data associated with top values in one or more columns.

In [ ]:
grouped_data.nlargest(10, columns = ['dollars_obligated_proportion'])

The following code block shows how we can *chain* methods to save the top 10 NAICS values to a list. However, since we had a multidimensional index, our list is a list of tuples.

In [ ]:
top_naics_codes = grouped_data.nlargest(10, columns = ['dollars_obligated_proportion']).index.tolist()
top_naics_codes

We will answer the remaining questions with respect to the top NAICS code. However, instead of hard-coding the NAICS code, we will just index the first value in the `top_naics_codes` list.

In [ ]:
index_val = 0

top_naics_codes[index_val]

Note that this returns a `tuple`.

In [ ]:
type(top_naics_codes[index_val])

We can get the NAICS code by accessing the first element of the tuple.

In [ ]:
top_naics_codes[index_val][0]

We can get the NAICS description by accessing the second element of the tuple.

In [ ]:
top_naics_codes[index_val][1]

The following code block saves these values as variables.

In [ ]:
# Get NAICS code and description
naics_code = top_naics_codes[index_val][0]
naics_description = top_naics_codes[index_val][1]

The following code block saves the data associated with the current NAICS code as a variable named `naics_data`.

In [ ]:
# Create a mask to subset the data
mask = data['naics_code'] == naics_code

# Get the data subset
naics_data = data[mask]

# Who is purchasing these products

The following code block determines the agencies with transactions associated with the target NAICS number and the total amount of money obligated.

In [ ]:
# Group the data
groupby_columns = ['awarding_agency_abbr', 'awarding_agency_name']

agg_dict = {
    'award_id_piid': ['nunique'],
    'total_dollars_obligated': ['sum'],
}

grouped_data = naics_data.groupby(groupby_columns).agg(agg_dict)

grouped_data.columns = ['_'.join(col).strip() for col in grouped_data.columns.values]

# Calculate the proportions
grouped_data['dollar_obligated_proportion'] = (grouped_data['total_dollars_obligated_sum']
                                               /grouped_data['total_dollars_obligated_sum'].sum())

grouped_data['awards_proportion'] = (grouped_data['award_id_piid_nunique']
                                     /grouped_data['award_id_piid_nunique'].sum())

# Print the data sorted by the number of awards
grouped_data.sort_values('awards_proportion')

Notice that we essentially copied and pasted the grouping and column calculation code from earlier in the cell above. Anytime you notice yourself doing so, look into defining a function. This is done in the following code block.

In [ ]:
def custom_grouper(df, agg_dict, groupby_columns):
    '''
    This function groups the provided DataFrame, df, by the columns
    specified in the groupby_columns argument. The aggregations specified
    in the agg_dict dictionary are applied. Also, each numeric column in the 
    aggregated DataFrame is used to create a proportion column. The aggregated data
    is returned as a DataFrame sorted by the keys of the agg_dict
    dictionary, in the order they are specified, i.e., first key
    has a higher sort priority than the second, etc...
    '''
    
    grouped_df = df.groupby(groupby_columns).agg(agg_dict)
    
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
    
    numeric_columns = grouped_df.select_dtypes(include='number').columns.tolist()

    for column in numeric_columns:
        grouped_df[f'{column}_proportion'] = (grouped_df[column]/grouped_df[column].sum())
        
    grouped_df = grouped_df.sort_values(numeric_columns)

    return grouped_df

The following code block shows how we can use the newly defined function.

In [ ]:
groupby_columns = ['awarding_agency_abbr', 'awarding_agency_name']

agg_dict = {
    'total_dollars_obligated': ['sum'], 
    'award_id_piid': ['nunique'],
}

custom_grouper(naics_data, agg_dict, groupby_columns)

# Who are we purchasing from?

The following code block determines the total amount and number of agencies assciated with transactions, grouped by the recipient.

In [ ]:
groupby_columns = ['recipient_duns', 'recipient_name']

agg_dict = {
    'awarding_agency_abbr': ['nunique'],
    'total_dollars_obligated': ['sum'],
}

custom_grouper(naics_data, agg_dict, groupby_columns)

# How are we purchasing?

The following code block determines the total amount associated with transactions, grouped by the contract pricing type.

In [ ]:
groupby_columns = ['type_of_contract_pricing', 
               'type_of_contract_pricing_code']

agg_dict = {
    'total_dollars_obligated': ['sum'],
}

custom_grouper(naics_data, agg_dict, groupby_columns)

The following code block determines the total amount associated with transactions, grouped by the solicitation procedures.

In [ ]:
groupby_columns = ['extent_competed', 
                   'solicitation_procedures']

agg_dict = {
    'total_dollars_obligated': ['sum'],
}

custom_grouper(naics_data, agg_dict, groupby_columns)

# All together

Let's now take everything we have done so far and automate the analysis for the 10 NAICS codes with the highest total dollars obligated. The following code block redefines our `custom_grouper` function. This is not necessary, but done simply to get everything in one place.

In [ ]:
def custom_grouper(df, agg_dict, groupby_columns):
    '''
    This function groups the provided DataFrame, df, by the columns
    specified in the groupby_columns argument. The aggregations specified
    in the agg_dict dictionary are applied. Also, each numeric column in the 
    aggregated DataFrame is used to create a proportion column. The aggregated data
    is returned as a DataFrame sorted by the keys of the agg_dict
    dictionary, in the order they are specified, i.e., first key
    has a higher sort priority than the second, etc...
    '''
    
    grouped_df = df.groupby(groupby_columns).agg(agg_dict)
    
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
    
    numeric_columns = grouped_df.select_dtypes(include='number').columns.tolist()

    for column in numeric_columns:
        grouped_df[f'{column}_proportion'] = (grouped_df[column]/grouped_df[column].sum())
        
    grouped_df = grouped_df.sort_values(numeric_columns)

    return grouped_df

The following code block specifies a simple function that we will use to print status messages during the automated analysis.

In [ ]:
def print_status(message, 
                 start_timestamp, 
                 current_timestamp, 
                 width = 60):
    '''
    A helper function to print a status message with elapsed time in seconds.
    '''
    
    print(f'{message} (elapsed time: {round(current_timestamp - start_timestamp, 2)} seconds)')
    print('-'*width)
    
    return

The following code block specifies the automated analysis. For each of the top 10 NAICS codes, an Excel file is written in the `outputs` folder with tabs for the various analysis steps.

In [ ]:
run_cell = True

if run_cell:
    import time
    start_time = time.time()
    
    output_folder_path = pathlib.Path('outputs')
    if not output_folder_path.exists():
        output_folder_path.mkdir(exist_ok = True)
    
    print_status(message = 'Reading data', 
                 start_timestamp = start_time, 
                 current_timestamp = time.time())
    
    data_filepath = pathlib.Path('data', 'AL_FY2019.csv')
    if data_filepath.exists():
        print('File exists, reading with pandas.')
        data = pd.read_csv(data_filepath)
    else:
        print('File does not exist!')
        
    print_status(message = 'Dropping non-positive values for total_dollars_obligated', 
                 start_timestamp = start_time, 
                 current_timestamp = time.time())
        
    mask = data['total_dollars_obligated'] <= 0
    data = data[~mask]

    
    print_status(message = 'Determining top NAICS codes by total_dollars_obligated', 
                 start_timestamp = start_time, 
                 current_timestamp = time.time())
    
    groupby_columns = ['naics_code', 
                       'naics_description']
    agg_dict = {
        'award_id_piid': ['nunique'], 
        'total_dollars_obligated': ['sum'],
    }

    grouped_data = custom_grouper(data, agg_dict, groupby_columns)

    top_naics_codes = grouped_data.nlargest(10, columns = ['total_dollars_obligated_sum']).index.tolist()

    for naics_code, naics_description in top_naics_codes:
        
        print_status(message = f'Starting analysis for NAICS code {int(naics_code)}', 
                 start_timestamp = start_time, 
                 current_timestamp = time.time())
        
        output_filepath = pathlib.Path('outputs', f'{int(naics_code)}_output.xlsx')
        with pd.ExcelWriter(output_filepath) as writer:  
            mask = data['naics_code'] == naics_code
            naics_data = data[mask]

            # Who is buying
            groupby_columns = ['awarding_agency_abbr', 
                               'awarding_agency_name']
            agg_dict = {
                'total_dollars_obligated': ['sum'],
                'award_id_piid': ['nunique'],
            }            
            custom_grouper(naics_data, agg_dict, groupby_columns).to_excel(writer, 'Buying Agencies')


            # Who are we buying from
            groupby_columns = ['recipient_duns', 
                               'recipient_name']
            agg_dict = {
                'awarding_agency_abbr': ['nunique'],
                'total_dollars_obligated': ['sum'],
            }
            custom_grouper(naics_data, agg_dict, groupby_columns).to_excel(writer, 'Recipients')

            
            # how are we buying - contracts
            groupby_columns = ['type_of_contract_pricing', 
                               'type_of_contract_pricing_code']
            agg_dict = {
                'total_dollars_obligated': ['sum'],
            }            
            custom_grouper(naics_data, agg_dict, groupby_columns).to_excel(writer, 'Contract Types')

            
            # how are we buying - solicitation
            groupby_columns = ['extent_competed', 
                               'solicitation_procedures']
            agg_dict = {
                'total_dollars_obligated': ['sum'],
            }
            custom_grouper(naics_data, agg_dict, groupby_columns).to_excel(writer, 'Solicitation Type')